# Simulation

Goal: to run a simulation based on **real-time location** tracking of ambulances. The simulation itself may or may not be used. But I hope that the **modules/ideas** making up this simulation could end up being useful.

- `Clocked`
- `Case events` are randomly entered

More information written below. 

In [1]:
import time
import sys
from enum import Enum

In [2]:
ambulance_count = 8

class Status(Enum):
    """ For ambulances """
    IDLE = 1
    ACTIVE = 2
    
amb_status = {} 
for ID in range(0, ambulance_count):
    amb_status[ID] = Status.IDLE

In [3]:
def timed_execute():
    """ Starts a clock """
    speed_times = 4
    run_time = 1
    start = time.time()
    time_diff = 0

    while True:
        """ This code makes the world run over time. """
        time.sleep(1/speed_times)
        
        print ("\rTime: " + str(time_diff))
        change_state()
        print()
        
        now = time.time()

        time_diff = (now - start)*speed_times
        time_diff = int(time_diff)
        
        if time_diff > run_time: break 

In [4]:
def change_state():
    global amb_status
    
    output = ""
    
    sys.stdout.flush()
    for ID in amb_status:
        output += ("Ambulance %i %s \n" %(ID, amb_status[ID]))
        
    sys.stdout.write(output)
        
    pass

In [5]:
timed_execute()

Time: 0
Ambulance 0 Status.IDLE 
Ambulance 1 Status.IDLE 
Ambulance 2 Status.IDLE 
Ambulance 3 Status.IDLE 
Ambulance 4 Status.IDLE 
Ambulance 5 Status.IDLE 
Ambulance 6 Status.IDLE 
Ambulance 7 Status.IDLE 

Time: 1
Ambulance 0 Status.IDLE 
Ambulance 1 Status.IDLE 
Ambulance 2 Status.IDLE 
Ambulance 3 Status.IDLE 
Ambulance 4 Status.IDLE 
Ambulance 5 Status.IDLE 
Ambulance 6 Status.IDLE 
Ambulance 7 Status.IDLE 



# Taking in Cases:
Okay, so now that I have clock, I want to implement reading in the cases that happen in "real" time. 

### Easy but False Simulation 

- One way I could do that is to read it from a file. Characteristics of the events would be:

    - `GPS Location`
    - `Severity`
    - `Time it comes in`

- Characteristics of the case that are unpredictable, but useful:

    - The `time` it takes to accomplish the case
    - The actual `travel time`

### Harder but Realistic Simulation

- The other thing I could do is to have the events happen `randomly`. This is not hard. 

- The last thing I could do is to `manually input` the events which is the most difficult since this would be a continuous program that reacts based on user inputs. It would probably use GRPCs. 

- The random events can be shaped by seeds, which means we may be able to `shape the way new events are initialized`. 

# Implementing the Simulation

- Ambulances should be timed out. When they are sent on cases, they are timed in. There is a `time t` the ambulance is on-case. When the time finishes, it returns to Idle.
- New cases are started when the `arguments` come in and the time matches its start date. The other way to start cases is based on real time: manually input it. 
- We can randomly generate the `timeline` of cases that will backbone the simulation.

# Pseudocode Algorithm

```
for each time unit (t):
    
    update all times.
    
    if a new case should start, then start it.
    
    if an
    

```